In [4]:
# tested in transformers==4.18.0 
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
import torch.nn.functional as F
import pandas as pd



In [8]:
dfs = pd.read_csv('ReportSplitSentences/testcsv.csv')
display(dfs)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/qjy/Desktop/FinTech/Project/Test/ReportSplitSentences/testcsv.csv'

In [30]:
def loadFinbertESG (df:pd):
    # Load the pre-trained model and tokenizer
    finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
    tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
    labels = ['None', 'Environmental', 'Social', 'Governance']
    for i,row in df.iterrows():
        sen = row['Sentences']
        inputs = tokenizer(sen, return_tensors='pt')
        outputs = finbert(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
        results = {label: prob.item() for label, prob in zip(labels, probs[0])}
        df.loc[i,'Env'] = results['Environmental']
        df.loc[i,'Soc'] = results['Social']
        df.loc[i,'Gov'] = results['Governance']
        df.loc[i,'None'] = results['None']
        print(f'{i}+:::{results}')
    return df

showresult = loadFinbertESG(dfs)
display(showresult)

/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0+:::{'Environmental': 0.993266224861145, 'Social': 0.001426791655831039, 'Governance': 0.0025409662630409002, 'Other': 0.002765921177342534}
1+:::{'Environmental': 0.9951030015945435, 'Social': 0.0012891615042462945, 'Governance': 0.0013555506011471152, 'Other': 0.0022523701190948486}
2+:::{'Environmental': 0.9943370223045349, 'Social': 0.001657078042626381, 'Governance': 0.0013353731483221054, 'Other': 0.0026704405900090933}
3+:::{'Environmental': 0.9900102019309998, 'Social': 0.003824761603027582, 'Governance': 0.0025128470733761787, 'Other': 0.0036522052250802517}
4+:::{'Environmental': 0.9937241673469543, 'Social': 0.0018323021940886974, 'Governance': 0.0017046657158061862, 'Other': 0.002738767070695758}
5+:::{'Environmental': 0.9947447776794434, 'Social': 0.001799214631319046, 'Governance': 0.0011862105457112193, 'Other': 0.002269754186272621}
6+:::{'Environmental': 0.9935745596885681, 'Social': 0.002570259850472212, 'Governance': 0.0016610663151368499, 'Other': 0.002194117754697

,Sentences,Env,Soc,Gov,Other
0,Item 1. Business Company Background The Comp...,0.993266,0.001427,0.002541,0.002766
1,The Company's fiscal year is the 52 or 53week ...,0.995103,0.001289,0.001356,0.002252
2,Products iPhone iPhone® is the Company's lin...,0.994337,0.001657,0.001335,0.002670
3,"The iPhone line includes iPhone 15 Pro, iPhone...",0.990010,0.003825,0.002513,0.003652
4,Mac Mac® is the Company's line of personal co...,0.993724,0.001832,0.001705,0.002739
...,...,...,...,...,...
512,"Directors, Executive Officers and Corporate Go...",0.867450,0.004152,0.005463,0.122935
513,Item 11.,0.943720,0.015384,0.023953,0.016943
514,Executive Compensation The information requir...,0.861282,0.003933,0.012036,0.122748
515,Item 12.,0.971363,0.006987,0.011381,0.010269


In [6]:
# Load the pre-trained model and tokenizer
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
  
text = 'Item 10. Directors, Executive Officers and Corporate GovernanceThe information required by this Item will be included in the Company\'s definitive proxy statement to be filed with the SEC within 120 days afterSeptember 30, 2023, in connection with the solicitation of proxies for theCompany\'s 2024 annual meeting of shareholders (the "2024 Proxy Statement"),and is incorporated herein by reference.'
# Tokenize the input text
inputs = tokenizer(text, return_tensors='pt')
# Get the model's raw logits
outputs = finbert(**inputs)

# Apply softmax to get probabilities
probs = F.softmax(outputs.logits, dim=-1)

# Define the labels (based on the FinBERT ESG model documentation)
labels = ['None', 'Environmental', 'Social', 'Governance']

# Map the probabilities to the labels
results = {label: prob.item() for label, prob in zip(labels, probs[0])}

print(results['Environmental']) # [{'label': 'Social', 'score': 0.9906041026115417}]
print(results)


0.003968365956097841
{'None': 0.8791201114654541, 'Environmental': 0.003968365956097841, 'Social': 0.005655191373080015, 'Governance': 0.1112564280629158}


In [34]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
 
tokenizer_name = "ESGBERT/GovernanceBERT-governance"
model_name = "ESGBERT/GovernanceBERT-governance"
 
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, max_len=512)
 
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer) # set device=0 to use GPU
 
# See https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline
print(pipe("Item 10. Directors, Executive Officers and Corporate Governance The information required by this Item will be included in the Company's definitive proxy statement to be filed with the SEC within 120 days afterSeptember 30, 2023, in connection with the solicitation of proxies for theCompany's 2024 annual meeting of shareholders (the '2024 Proxy Statement'),and is incorporated herein by reference.", padding=True, truncation=True))


[{'label': 'governance', 'score': 0.66694575548172}]


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
finbert1 = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg' ,num_labels=4, return_dict=True)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
nlp = pipeline("text-classification", model=finbert1, tokenizer=tokenizer,return_all_scores=True)
text = 'Item 10. Directors, Executive Officers and Corporate GovernanceThe information required by this Item will be included in the Company\'s definitive proxy statement to be filed with the SEC within 120 days afterSeptember 30, 2023, in connection with the solicitation of proxies for theCompany\'s 2024 annual meeting of shareholders (the "2024 Proxy Statement"),and is incorporated herein by reference.'
results = nlp(text)
print(results)
print(results['None'])
print(results['Environmental'])
print(results['Social'])
print(results['Governance'])


/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'None', 'score': 0.8791201114654541}, {'label': 'Environmental', 'score': 0.003968365956097841}, {'label': 'Social', 'score': 0.005655191373080015}, {'label': 'Governance', 'score': 0.1112564280629158}]]


TypeError: list indices must be integers or slices, not str

In [7]:
# tested in transformers==4.18.0 
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
text = 'Item 10. Directors, Executive Officers and Corporate GovernanceThe information required by this Item will be included in the Company\'s definitive proxy statement to be filed with the SEC within 120 days afterSeptember 30, 2023, in connection with the solicitation of proxies for theCompany\'s 2024 annual meeting of shareholders (the "2024 Proxy Statement"),and is incorporated herein by reference.'

results = nlp(text)
print(results) # [{'label': 'Social', 'score': 0.9906041026115417}]


/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'label': 'None', 'score': 0.8791201114654541}]
